In [3]:
import numpy as np
import pandas as pd

# ========= One-sided HP via Kalman filter =========
def one_sided_hp_trend_kalman(x: np.ndarray, lam: float) -> np.ndarray:
    """
    一侧 HP 趋势（卡尔曼滤波实现）
    模型：
        y_t = τ_t + ε_t, ε_t ~ N(0, σ_eps^2)
        τ_t = τ_{t-1} + β_{t-1}
        β_t = β_{t-1} + ζ_t, ζ_t ~ N(0, σ_zeta^2)
    并满足 σ_eps^2 / σ_zeta^2 = λ（HP 约束），取 σ_eps^2=1，σ_zeta^2=1/λ。
    返回：每个 t 的滤波态 τ_t|t（只用到 t 期以前信息），即 one-sided 趋势。
    """
    x = np.asarray(x, dtype=float).reshape(-1)
    n = x.size

    sigma_eps2 = 1.0
    sigma_zeta2 = 1.0 / lam

    # 状态方程/观测方程
    F = np.array([[1.0, 1.0],
                  [0.0, 1.0]])         # transition
    H = np.array([[1.0, 0.0]])         # observation
    Q = np.array([[0.0, 0.0],
                  [0.0, sigma_zeta2]]) # process noise
    R = np.array([[sigma_eps2]])       # obs noise

    # 扩散先验
    a = np.array([0.0, 0.0])           # state mean
    P = np.eye(2) * 1e6                # large variance

    trend = np.zeros(n, dtype=float)

    for t in range(n):
        # 预测
        a_pred = F @ a
        P_pred = F @ P @ F.T + Q

        # 创新
        y_pred = H @ a_pred
        v = x[t] - y_pred[0]
        S = H @ P_pred @ H.T + R  # 1x1

        # 卡尔曼增益
        K = (P_pred @ H.T) / S[0,0]  # 2x1

        # 更新
        a = a_pred + (K.flatten() * v)
        P = P_pred - K @ H @ P_pred

        # 当期一侧趋势
        trend[t] = a[0]

    return trend

def one_sided_hp_filter(x: np.ndarray, lam: float) -> np.ndarray:
    # 与 MATLAB 同名：返回“趋势”，外面做 x - trend 得到周期分量
    return one_sided_hp_trend_kalman(x, lam)

def demean(x: np.ndarray) -> np.ndarray:
    mu = np.nanmean(x)
    return x - mu

# ========= 读取数据 =========
input_path = "SW_Data4HW.xlsx"  # 你上传的文件
sheet_name = "Sheet1"

df_in = pd.read_excel(input_path, sheet_name=sheet_name)

# 取前 7 列：c i y h pi rw r（按位置，不依赖列名）
dataQ = df_in.iloc[:, :8].to_numpy(float)

# # ========= 时间轴 1965Q1 -- 2024Q4 =========
# n = dataQ.shape[0]
# timeline = 1965.0 + 0.1 * np.arange(n)  # 1965.0, 1965.25, ...

# λ = 1600（季频）
lam = 1600.0

# ========= 计算周期成分 / 去均值 =========
c_trend  = one_sided_hp_filter(dataQ[:, 1], lam)
i_trend  = one_sided_hp_filter(dataQ[:, 2], lam)
y_trend  = one_sided_hp_filter(dataQ[:, 3], lam)
h_trend  = one_sided_hp_filter(dataQ[:, 4], lam)
rw_trend = one_sided_hp_filter(dataQ[:, 6], lam)

timeline = dataQ[:, 0]
cobs   = dataQ[:, 1] - c_trend
iobs   = dataQ[:, 2] - i_trend
yobs   = dataQ[:, 3] - y_trend
labobs = dataQ[:, 4] - h_trend
piobs  = demean(dataQ[:, 5])
rwobs  = dataQ[:, 6] - rw_trend
robs   = demean(dataQ[:, 7])

# ========= 输出到 CK08_Q6.xlsx =========
out_df = pd.DataFrame({
    "timeline": timeline,
    "cobs": cobs,
    "iobs": iobs,
    "yobs": yobs,
    "labobs": labobs,
    "piobs": piobs,
    "rwobs": rwobs,
    "robs": robs
})

output_path = "CK08_Q6.xlsx"  # 当前工作目录；CK08_Q6.xlsx
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    out_df.to_excel(writer, sheet_name="Sheet1", index=False)

print(f"Saved: {output_path}")


Saved: CK08_Q6.xlsx
